In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
from importlib import reload


#import my personal modules created for this report
import wrangle_zillow as w
#import explore as e
#import model as m

## Acquire and Summarize
### 1. Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. 
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) 
- Only include properties that have a latitude and longitude value.

In [2]:
sql_query = '''
    SELECT *
    FROM predictions_2017 
    LEFT JOIN properties_2017 USING (parcelid)
    LEFT JOIN airconditioningtype USING (airconditioningtypeid)
    LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
    LEFT JOIN buildingclasstype USING (buildingclasstypeid)
    LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
    LEFT JOIN propertylandusetype USING (propertylandusetypeid)
    LEFT JOIN storytype USING (storytypeid)
    LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
    WHERE YEAR(transactiondate) = 2017
    AND latitude IS NOT NULL
    AND longitude IS NOT NULL;
    '''

df = pd.read_sql(sql_query, w.get_db_url(db = 'zillow'))

In [3]:
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,logerror,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,0,0.025595,...,None,NaN,6.059063e+13,None,None,None,None,Single Family Residential,None,None
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1,0.055619,...,None,NaN,6.111001e+13,None,None,None,None,Single Family Residential,None,None
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,2,0.005383,...,None,NaN,6.059022e+13,None,None,None,None,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,3,-0.103410,...,None,NaN,6.037300e+13,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,4,0.006940,...,None,NaN,6.037124e+13,Central,None,None,Central,Condominium,None,None


In [4]:
df.shape

(77579, 69)

In [5]:
dups = df[df.duplicated(subset='parcelid', keep='last')].index

In [6]:
df.drop(dups, inplace=True)

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [7]:
df.shape

(77381, 69)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77381 non-null  float64
 3   heatingorsystemtypeid         49440 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      206 non-null    float64
 6   airconditioningtypeid         24953 non-null  float64
 7   parcelid                      77381 non-null  int64  
 8   id                            77381 non-null  int64  
 9   logerror                      77381 non-null  float64
 10  transactiondate               77381 non-null  object 
 11  id                            77381 non-null  int64  
 12  basementsqft                  50 non-null     float64
 13  b

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
typeconstructiontypeid,222.0,6.040541e+00,5.572847e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.300000e+01
storytypeid,50.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,77381.0,2.618263e+02,5.141199e+00,3.100000e+01,2.610000e+02,2.610000e+02,2.660000e+02,2.750000e+02
heatingorsystemtypeid,49440.0,3.920510e+00,3.592779e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,24953.0,1.813289e+00,2.967894e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
parcelid,77381.0,1.300715e+07,3.481346e+06,1.071186e+07,1.153830e+07,1.253157e+07,1.421183e+07,1.676893e+08
id,77381.0,3.884789e+04,2.240216e+04,0.000000e+00,1.946000e+04,3.886900e+04,5.825200e+04,7.761200e+04
logerror,77381.0,1.662499e-02,1.701908e-01,-4.655420e+00,-2.437679e-02,6.626972e-03,3.920313e-02,5.262999e+00


In [10]:
for column in df.columns:
    print(column)
    print(df[column].value_counts())
    print('\n ------------------------- \n')

typeconstructiontypeid
6.0     219
13.0      1
10.0      1
4.0       1
Name: typeconstructiontypeid, dtype: int64

 ------------------------- 

storytypeid
7.0    50
Name: storytypeid, dtype: int64

 ------------------------- 

propertylandusetypeid
261.0    52320
266.0    19294
246.0     2009
269.0     1944
248.0      727
247.0      535
265.0      333
263.0       74
275.0       58
260.0       37
267.0       29
31.0        15
264.0        6
Name: propertylandusetypeid, dtype: int64

 ------------------------- 

heatingorsystemtypeid
2.0     33550
7.0     14065
24.0      846
6.0       776
20.0      104
13.0       59
18.0       23
1.0        13
10.0        3
11.0        1
Name: heatingorsystemtypeid, dtype: int64

 ------------------------- 

buildingclasstypeid
4.0    14
3.0     1
Name: buildingclasstypeid, dtype: int64

 ------------------------- 

architecturalstyletypeid
7.0     172
8.0      19
21.0      7
2.0       5
3.0       3
Name: architecturalstyletypeid, dtype: int64

 -------

ValueError: Grouper for 'id' not 1-dimensional

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [11]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    percent_missing = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': percent_missing})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [12]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingclassdesc,77366,99.980615
buildingclasstypeid,77366,99.980615
finishedsquarefeet13,77340,99.947015
basementsqft,77331,99.935385
storydesc,77331,99.935385
...,...,...
roomcnt,0,0.000000
regionidcounty,0,0.000000
rawcensustractandblock,0,0.000000
bathroomcnt,0,0.000000


## Prepare

### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [17]:
df.columns.to_list()

['typeconstructiontypeid',
 'storytypeid',
 'propertylandusetypeid',
 'heatingorsystemtypeid',
 'buildingclasstypeid',
 'architecturalstyletypeid',
 'airconditioningtypeid',
 'parcelid',
 'id',
 'logerror',
 'transactiondate',
 'id',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'threequarterbathnbr',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqf

In [13]:
df.propertylandusedesc.unique() 

array(['Single Family Residential', 'Condominium',
       'Duplex (2 Units, Any Combination)', 'Residential General',
       'Planned Unit Development', 'Triplex (3 Units, Any Combination)',
       'Quadruplex (4 Units, Any Combination)', 'Cluster Home',
       'Commercial/Office/Residential Mixed Used', 'Cooperative',
       'Manufactured, Modular, Prefabricated Homes', 'Mobile Home',
       'Townhouse'], dtype=object)

Single family homes include
```Manufactured, Modular, Prefabricated Homes```
```Single Family Residential```
```Condominium```
```Cluster Home```
```Mobile Home```
```Townhouse```

**Based on ```calculatedfinishedsquarefeet``` and ```taxvaluedollarcnt```

In [34]:
df[df.propertylandusedesc=='Townhouse'].calculatedfinishedsquarefeet.mean()

1405.5

In [35]:
df = df[(df.propertylandusedesc=='Manufactured, Modular, Prefabricated Homes') | 
        (df.propertylandusedesc=='Single Family Residential') |
        (df.propertylandusedesc=='Condominium') |
        (df.propertylandusedesc=='Cluster Home') |
        (df.propertylandusedesc=='Mobile Home') |
        (df.propertylandusedesc=='Townhouse')]

In [36]:
df

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,logerror,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,0,0.025595,...,None,NaN,6.059063e+13,None,None,None,None,Single Family Residential,None,None
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1,0.055619,...,None,NaN,6.111001e+13,None,None,None,None,Single Family Residential,None,None
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,2,0.005383,...,None,NaN,6.059022e+13,None,None,None,None,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,3,-0.103410,...,None,NaN,6.037300e+13,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,4,0.006940,...,None,NaN,6.037124e+13,Central,None,None,Central,Condominium,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10833991,77608,-0.002245,...,None,NaN,6.037132e+13,Central,None,None,Central,Condominium,None,None
77575,NaN,NaN,261.0,2.0,NaN,NaN,NaN,11000655,77609,0.020615,...,None,NaN,6.037101e+13,None,None,None,Central,Single Family Residential,None,None
77576,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17239384,77610,0.013209,...,None,NaN,6.111008e+13,None,None,None,None,Single Family Residential,None,None
77577,NaN,NaN,261.0,2.0,NaN,NaN,1.0,12773139,77611,0.037129,...,None,NaN,6.037434e+13,Central,None,None,Central,Single Family Residential,None,None


### 2. Create a function that will drop rows or columns based on the percent of values that are missing: ```handle_missing_values(df, prop_required_column, prop_required_row)```.

In [37]:
def handle_missing_values(df, prop_required_columns=0.5, prop_required_rows=0.75):
    column_threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=column_threshold)
    row_threshold = int(round(prop_required_rows * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=row_threshold)
    return df

In [38]:
handle_missing_values(df)

,propertylandusetypeid,heatingorsystemtypeid,parcelid,id,logerror,transactiondate,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,...,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemdesc,propertylandusedesc
0,261.0,NaN,14297519,0,0.025595,2017-01-01,1727539,3.5,4.0,NaN,...,NaN,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,None,Single Family Residential
1,261.0,NaN,17052889,1,0.055619,2017-01-01,1387261,1.0,2.0,NaN,...,NaN,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,None,Single Family Residential
2,261.0,NaN,14186244,2,0.005383,2017-01-01,11677,2.0,3.0,NaN,...,NaN,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,None,Single Family Residential
3,261.0,2.0,12177905,3,-0.103410,2017-01-01,2288172,3.0,4.0,8.0,...,1.0,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,Central,Single Family Residential
4,266.0,2.0,10887214,4,0.006940,2017-01-01,1970746,3.0,3.0,8.0,...,1.0,1964.0,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,266.0,2.0,10833991,77608,-0.002245,2017-09-20,2864704,3.0,3.0,8.0,...,1.0,1980.0,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,Central,Condominium
77575,261.0,2.0,11000655,77609,0.020615,2017-09-20,673515,2.0,2.0,6.0,...,1.0,1940.0,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,Central,Single Family Residential
77576,261.0,NaN,17239384,77610,0.013209,2017-09-21,2968375,2.0,4.0,NaN,...,NaN,1964.0,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,None,Single Family Residential
77577,261.0,2.0,12773139,77611,0.037129,2017-09-21,1843709,1.0,3.0,4.0,...,1.0,1954.0,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,Central,Single Family Residential


In [41]:
df.columns.to_list()

['typeconstructiontypeid',
 'storytypeid',
 'propertylandusetypeid',
 'heatingorsystemtypeid',
 'buildingclasstypeid',
 'architecturalstyletypeid',
 'airconditioningtypeid',
 'parcelid',
 'id',
 'logerror',
 'transactiondate',
 'id',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'threequarterbathnbr',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqf

In [48]:
ids_to_drop = ['airconditioningtypeid','architecturalstyletypeid','buildingclasstypeid','heatingorsystemtypeid','propertylandusetypeid','storytypeid','typeconstructiontypeid']

df = df.drop(columns=ids_to_drop)

In [49]:
df.shape

(72085, 62)